Boilerplate setup for imports and MySQL connection

Documentation on connecting to a local database:https://dev.mysql.com/doc/workbench/en/wb-getting-started-tutorial-create-connection.html

*Should be done with connection verified before anything else*

Helpful link for PyMySql setup: https://www.tutorialspoint.com/python3/python_database_access.htm

(Dr. Bishop may want to set this up differently for students working on the project)

In [49]:
#Make sure MySQL is installed following guidelines of tutorial above before executing this code
import pymysql
import string
import numpy as np
import sqlite3
import json

from __future__ import print_function

import pymysql


conn = pymysql.connect(host='localhost', port=3306, user='root', passwd='root', db='thr2013')

cur = conn.cursor()

cursor(*SQL statement here*) is a helper function used reduce code and increase readability

Read about SQL cursors here: https://www.tutorialspoint.com/plsql/plsql_cursors.htm

In [50]:
def cursor(query):
    cur.execute(query)
    return cur
#note cursor returns query set as "cur"

Check to make sure cursor function is working properly...

fetchone() method retrieves next row of a query result set and returns a single sentence

Each book is associated with a unique id. This query returns id 26

In [84]:
cursor("select * from wpreader_book_search where id = 26").fetchone()

(26,
 'Jimmy Choo Shoes Jimmy Choo shoes can be red. Jimmy Choo shoes can be black. Jimmy Choo shoes can be green. Jimmy Choo shoes can be white. Jimmy Choo shoes can be hot pink. Jimmy Choo shoes can be pink. Jimmy Choo shoes can be yellow. Jimmy Choo shoes can be gold. Jimmy Choo shoes can be silver. Jimmy Choo makes shoes! fashion shoes Karen',
 '{"title":"Jimmy Choo Shoes","author":"Karen","type":"T","audience":"E","reviewed":true,"language":"en","tags":["fashion","shoes"],"categories":["Fict"],"pages":[{"text":"Jimmy Choo Shoes","url":"\\/cache\\/images\\/24\\/2312776324_a915bcbea7_t.jpg","width":67,"height":100},{"text":"Jimmy Choo shoes can be red.","url":"\\/cache\\/images\\/24\\/2312776324_a915bcbea7.jpg","width":335,"height":500},{"text":"Jimmy Choo shoes can be black.","url":"\\/cache\\/images\\/48\\/67276348_80da8587d7.jpg","width":500,"height":441},{"text":"Jimmy Choo shoes can be green.","url":"\\/cache\\/images\\/29\\/88673329_2c0e4b34b0.jpg","width":500,"height":333},{"

*Note*: We will explain the basic algorithm used in comments of code but for further explanation look up "bag of words algorithm". This is a basic implementation of b.o.w.

build_vocab_books constructs and return two dictionaries: books and vocab.

books uses book ids as keys and arrays of individual words as keys

vocab uses words as keys and word occurance count as values. Note a word must occur at least 3 times to be included

Also note that since there is not a direct translation of ids from the sqlite db to the MySQL db, the same books from each db must be connected by "slug" which is like the titular identifier of a book. This is used later on in the construction of the suggestions table.

In [86]:
#In the test code the following two dictionaries are left out of the following function so that they can be used later..
#..on without having to call this funciton. You will want to have this function return these dicts in actual..
#..implementation
id_to_slug = {}
slug_to_id = {}
def build_vocab_books():
    #translator helps strip strings of punctuation
    translator = str.maketrans({key: None for key in string.punctuation})
    
    vocab = {}
    books = {}
    
    #setting our cursor
    #simply change number following limit to expand range for testing. Actual implementation should not include limit
    cursor("select id,content,json from wpreader_book_search limit 100")
    
    #loops through cursor, fills books dict, key - book ids "ids", value - book body "content"
    #notice we are utilizing the json library to decode the json from each book into a dict..
    #..so as to acces the slug by the key "slug"
    for ids,content,cur_json in cur:
        books[ids] = content
        cur_json_dict = json.loads(cur_json)
        id_to_slug[ids] = cur_json_dict['slug']
        slug_to_id[cur_json_dict['slug']] = ids
        
    
    #loops through books dict breaking up book body to individual words
    for ids,content in books.items():
        #normalize and split the body, store for use in vocab
        book_body = content.lower().translate(translator).split()
        
        books[ids] = book_body
        
        #populates vocab dict and tallies occurances
        #nested to loop through each string
        for word in book_body:
            if word in vocab:
                vocab[word] += 1
            else:
                vocab[word] = 1
    
    #need to use a separate array to store words we want to delete
    #so that the dict doesn't change as the loop runs
    sparse_words = []
    for word,count in vocab.items():
        if count < 3:
            sparse_words.append(word)
    
    #loops through sparse_words and deletes words from vocab
    for word in sparse_words:
        del vocab[word]
    
    return books,vocab
build_vocab_books()

({26: ['jimmy',
   'choo',
   'shoes',
   'jimmy',
   'choo',
   'shoes',
   'can',
   'be',
   'red',
   'jimmy',
   'choo',
   'shoes',
   'can',
   'be',
   'black',
   'jimmy',
   'choo',
   'shoes',
   'can',
   'be',
   'green',
   'jimmy',
   'choo',
   'shoes',
   'can',
   'be',
   'white',
   'jimmy',
   'choo',
   'shoes',
   'can',
   'be',
   'hot',
   'pink',
   'jimmy',
   'choo',
   'shoes',
   'can',
   'be',
   'pink',
   'jimmy',
   'choo',
   'shoes',
   'can',
   'be',
   'yellow',
   'jimmy',
   'choo',
   'shoes',
   'can',
   'be',
   'gold',
   'jimmy',
   'choo',
   'shoes',
   'can',
   'be',
   'silver',
   'jimmy',
   'choo',
   'makes',
   'shoes',
   'fashion',
   'shoes',
   'karen'],
  29: ['my',
   'birthday',
   'at',
   'my',
   'birthday',
   'i',
   'want',
   'balloons',
   'at',
   'my',
   'birthday',
   'i',
   'want',
   'a',
   'cake',
   'at',
   'my',
   'birthday',
   'i',
   'want',
   'some',
   'friends',
   'at',
   'my',
   'birthday'

So now we have created a way to associate a book's id with all of its individual words and we have created an index of words from the corpus of our query that meet our word count requirement. Both of these are integral to the next step.

We must now use our last block to build vectors out of each book. Each word from our vocab dict will have a dimension in every book's vector. Whether a dimension is filled or not in each book vector is determined by that given book's word count for said dimension's word.

Please execute the code below for an example. Note the normalization of vectors, so if you would like to see integer values for a more intuitive view just comment out the normalization line

In [87]:
def book_to_vec():
    
    #word_to_index dict keeps track of which word is assigned to which dimension
    word_to_index = {}
    
    #assigning dicts returned from build_vocab_books
    books,vocab = build_vocab_books()
    
    #This is generating an enumeration for every word in the corpus of books
    #Keeps track of which word is assigned to which dimension
    for idx, word in enumerate(sorted(vocab)): 
        word_to_index[word] = idx
    
    #book_to_vec uses a book's id as key and its word count vector as value
    book_to_vec = {}
    
    #Looping through the IDs and individual word arrays associated with each book
    for bookID,body in books.items(): 
        
        #Filling book_to_vec, key - book ID, value - zero vector size of vocab
        #We will fill zero vector in nested loop below
        book_to_vec[bookID] = np.zeros(len(vocab)) 
        
        #looping through each word
        for word in body:
            #This if condition makes sure we are only adding words that meet word count requirement
            if word in vocab:
                #populates the zero vectors by counting occurances of words and increments...
                #...the corresponding dimension associated with a given word
                #wti stores the dimension number associated with a given word
                wti = word_to_index[word]
                #Accesses the book vector for a given book then accesses dimension wti and increments
                book_to_vec[bookID][wti] += 1
        #normalizes the book vectors to make each book vector a unit vector
        book_to_vec[bookID] = book_to_vec[bookID]/np.sqrt(np.dot(book_to_vec[bookID], book_to_vec[bookID]))
                
    return book_to_vec
 
book_to_vec()

{26: array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.38100038,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.04233338,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.38100038,  0.        ,
         0.        ,  0.        ,  0.        , 

Now that we have each books word count vector we can finally compute the textual "similarity" between each book. This is accomplished by computing the dot product between each vector. Picture each book's vector in a shared n dimensional space. Using bag of words, the smaller the angle between any two vectors the more similar they are. Dot product ranges from -1 to 1, with a higher value meaning a smaller angle.

For a more thorough explanation: https://en.wikipedia.org/wiki/Dot_product

similarity() constructs the dictionary that we will draw from to help populate the suggetion table

In [88]:
def similarity():
    
    #most_similar is the dict we will return
    #Uses book1's id as its key and the top-ten most similar books as its value
    #So each book is associated with an array of ten two tuples wherein the tuple is as follows...
    #...(dot-product, book2 id)
    most_similar = {}
    
    #
    book_to_vector = book_to_vec()
    
    #Starting with book1 for comparison
    for book1, book1_vec in book_to_vector.items():
        #setting book id as key with an empty list as value
        most_similar[book1] = []
        
        #now loop through all possible book2s and compute dot product between both books' vectors
        for book2, book2_vec in book_to_vector.items():
            #any book is the most similar with itself and we don't want that
            if book1 != book2:
                #assigning x to be the dot product between book1 and book2
                x = np.dot(book1_vec, book2_vec)
               
                #appends book2 to list associated with book1
                most_similar[book1].append((x, book2))
                
                #immediately sorts from highest to lowest dot product
                most_similar[book1].sort(reverse = True)
                
                #pops from list so that only ten recommendations are kept in list at a time
                if len(most_similar[book1]) > 10:
                    most_similar[book1].pop()
    return most_similar
similarity()

{26: [(0.3968109183932873, 134),
  (0.35921060405354982, 121),
  (0.29459558752263287, 111),
  (0.28138742976632514, 166),
  (0.24906050024681917, 116),
  (0.20640175151740603, 157),
  (0.20307388540270591, 189),
  (0.15659019199862312, 88),
  (0.14483461283656418, 73),
  (0.14408767870768324, 195)],
 29: [(0.45000878932000049, 192),
  (0.32287563632221267, 161),
  (0.32227850093563321, 201),
  (0.31357598440718421, 136),
  (0.31193682406898132, 100),
  (0.29798707686910286, 130),
  (0.28745999044971693, 175),
  (0.27952467385196555, 145),
  (0.26643834682594919, 200),
  (0.26510255052225268, 165)],
 69: [(0.14896989288278101, 195),
  (0.12756379785237865, 88),
  (0.1242625304369271, 136),
  (0.11611694995742754, 98),
  (0.11094003924504582, 197),
  (0.097166008154370043, 73),
  (0.095346258924559224, 154),
  (0.08796644381862459, 153),
  (0.082841686957951396, 140),
  (0.0802044171976313, 141)],
 71: [(0.1484575537959652, 148),
  (0.10843201340436633, 195),
  (0.099265783723903447, 77

The next metric of similarity we use is a Jaccard index (https://en.wikipedia.org/wiki/Jaccard_index). To get similarity using the Jaccard index, we construct a set for each book that is the non-bot IP addresses that have read each of the books. 

All of this information is in the SQLite dump of reader data that is created each day. 

The following code results in a dict which maps book slugs to their 10 most similar book slugs along with the similarity metric. 

Note: this may need to be reworked. Doing so many queries can take an extremely large amount of time. 

In [89]:
sqlite_conn = sqlite3.connect('db.sqlite') # requires the sqlite read data dump that is created daily
sqlite_cur = sqlite_conn.cursor()

def get_ids_and_slugs():
    sqlite_cur.execute("select id,slug from books")
    return [(a,b) for (a,b) in sqlite_cur.fetchall()]

get_book_ip_set_cache = {}
def get_book_ip_set(book_id): # cache is used to avoid redundant queries
    if book_id in get_book_ip_set_cache:
        return get_book_ip_set_cache[book_id]
    else:
        sqlite_cur.execute("""SELECT ip from read where book = {} and ip not in (select * from bots)""".format(book_id))
        rv = set([a for (a,) in sqlite_cur.fetchall()])
        get_book_ip_set_cache[book_id] = rv
        return rv

get_jaccard_similarity_cache = {} # very simple dynamic programming approach to try to reduce query number
def get_jaccard_similarity(book1_id, book2_id):
    if (book1_id, book2_id) in get_jaccard_similarity_cache:
        return get_jaccard_similarity_cache[(book1_id, book2_id)]
    book_1_ip = get_book_ip_set(book1_id)
    book_2_ip = get_book_ip_set(book2_id)
    # return the definition of jaccard index, size of intersection of the sets over the size of the union
    denom = float(len(book_1_ip | book_2_ip)) 
    if denom == 0: 
        get_jaccard_similarity_cache[(book1_id, book2_id)] = 0
        get_jaccard_similarity_cache[(book2_id, book1_id)] = 0
        return 0
    else:
        rv = float(len(book_1_ip & book_2_ip)) / denom
        get_jaccard_similarity_cache[(book1_id, book2_id)] = rv
        get_jaccard_similarity_cache[(book2_id, book1_id)] = rv
        return rv
sqlite_cur.execute("select * from books where id = 26")
sqlite_cur.fetchall()

[(26, '1-2-3-dance', 9)]

In [90]:
def jaccard_similarity():
    # very similar to "similarity" function created earlier
    # returns a dict of ids to their 10 most similar book slugs with the similarity value
    id_slug = get_ids_and_slugs()[:20] # remove 
    most_similar = {}
    for book1_id, book1_slug in id_slug:
        most_similar[book1_slug] = []
        for book2_id, book2_slug in id_slug:
            if book1_slug != book2_slug:
                most_similar[book1_slug].append((get_jaccard_similarity(book1_id,book2_id), book2_slug))
                most_similar[book1_slug].sort(reverse = True)
                if len(most_similar[book1_slug]) > 10:
                    most_similar[book1_slug].pop() # only keep the 10 most similar
    return most_similar
jaccard_similarity()

{'a-rainbow-of-hats': [(0.10606060606060606, 'noodles'),
  (0.08426966292134831, 'pizza'),
  (0.07210031347962383, 'flour-dough-bread'),
  (0.06483300589390963, 'my-birthday'),
  (0.05934718100890208, 'haircut-time'),
  (0.04984423676012461, 'strange-planet'),
  (0.04960835509138381, 'jimmy-choo-shoes'),
  (0.03485254691689008, 'cars'),
  (0.033936651583710405, 'the-book-of-bikes'),
  (0.03125, 'dogs')],
 'birds': [(0.0531496062992126, 'my-birthday'),
  (0.047619047619047616, 'strange-planet'),
  (0.0440771349862259, 'cars'),
  (0.04119850187265917, 'frogs'),
  (0.036544850498338874, 'jeff-gordon'),
  (0.03076923076923077, 'flour-dough-bread'),
  (0.029891304347826088, 'pizza'),
  (0.027522935779816515, 'dogs'),
  (0.025974025974025976, 'jimmy-choo-shoes'),
  (0.021505376344086023, 'noodles')],
 'cars': [(0.07159353348729793, 'pizza'),
  (0.06786427145708583, 'the-book-of-bikes'),
  (0.06683804627249357, 'strange-planet'),
  (0.06430155210643015, 'jimmy-choo-shoes'),
  (0.0562180579216

Lastly we have the suggestion table generation

Though this code will work, we leave this largely up to the implementors to ensure code correctness

In [91]:
cur.execute("""
    DROP TABLE if exists suggestions
""")
#Currently recreating the suggestions table everytime the batch job is ran
#As can be seen below there will be 5 columns to the suggestion table
cur.execute("""
    CREATE TABLE IF NOT EXISTS suggestions (
        id integer primary key not null AUTO_INCREMENT,
        book1_id integer, 
        book2_id integer,
        jaccard_similarity float,
        word_similarity float
    )
""")

#Helper function for inserting new entries into suggestions table
#Using .format() for dynamic varibles is important
def make_suggestion(book_id1, book_id2, jaccard_similarity, word_similarity):
    cur.execute("""INSERT INTO suggestions (book1_id, 
                                            book2_id, 
                                            jaccard_similarity, 
                                            word_similarity)
                                values ({}, {}, {}, {})""".format(book_id1, book_id2, jaccard_similarity, word_similarity))
#This helper function offers two paths - insert or update
def add_update_jaccard(book_id1, book_id2, jaccard_similarity):
    cur.execute("select * from suggestions where book1_id = {} and book2_id = {}".format(book_id1, book_id2))
    result = cur.fetchall()
    if len(result) == 0:
        make_suggestion(book_id1, book_id2, jaccard_similarity, 0)
    else:
        cur.execute("""
            UPDATE suggestion
                SET jaccard_similarity = {}
                WHERE book1_id = {} and book2_id = {}
        """.format(jaccard_similarity, book_id1, book_id2))
#Same thing but for pulling for bag of words
def add_update_word(book_id1, book_id2, word_similarity):
    cur.execute("select * from suggestions where book1_id = {} and book2_id = {}".format(book_id1, book_id2))
    res = cur.fetchall()
    if len(res) == 0:
        make_suggestion(book_id1, book_id2, 0, word_similarity)
    else:
        cur.execute("""
            UPDATE suggestion
                SET word_similarity = {}
                WHERE book1_id = {} and book2_id = {}
        """.format(word_similarity, book_id1, book_id2))

A quick example of how to populate this table given the methods we have above

Note: this block will return a key error while testing code with a limit on the query set. This means that the add_update_jaccard will not properly run until the slug_to_id dict has been fully realized

In [92]:
def populate_similarity_table():
    word_similarity = similarity()
    for ids, suggestions in word_similarity.items():
        for word_sim, id2 in suggestions:
            add_update_word(ids, id2, word_sim)
    jac_similarity = jaccard_similarity()
    for slug, suggestions in jac_similarity.items():
        for jaccard_sim, slug2 in suggestions:
            id1 = slug_to_id[slug]
            id2 = slug_to_id[slug2]
            add_update_jaccard(id1, id2, jaccard_sim)

populate_similarity_table()       

KeyError: 'dogs'

The following is an example of what a .fetchall() should look like

In [94]:
cur.execute("""
    select * from suggestions
""")
cur.fetchall()

((1, 26, 134, 0.0, 0.396811),
 (2, 26, 121, 0.0, 0.359211),
 (3, 26, 111, 0.0, 0.294596),
 (4, 26, 166, 0.0, 0.281387),
 (5, 26, 116, 0.0, 0.24906),
 (6, 26, 157, 0.0, 0.206402),
 (7, 26, 189, 0.0, 0.203074),
 (8, 26, 88, 0.0, 0.15659),
 (9, 26, 73, 0.0, 0.144835),
 (10, 26, 195, 0.0, 0.144088),
 (11, 29, 192, 0.0, 0.450009),
 (12, 29, 161, 0.0, 0.322876),
 (13, 29, 201, 0.0, 0.322278),
 (14, 29, 136, 0.0, 0.313576),
 (15, 29, 100, 0.0, 0.311937),
 (16, 29, 130, 0.0, 0.297987),
 (17, 29, 175, 0.0, 0.28746),
 (18, 29, 145, 0.0, 0.279525),
 (19, 29, 200, 0.0, 0.266438),
 (20, 29, 165, 0.0, 0.265103),
 (21, 69, 195, 0.0, 0.14897),
 (22, 69, 88, 0.0, 0.127564),
 (23, 69, 136, 0.0, 0.124263),
 (24, 69, 98, 0.0, 0.116117),
 (25, 69, 197, 0.0, 0.11094),
 (26, 69, 73, 0.0, 0.097166),
 (27, 69, 154, 0.0, 0.0953463),
 (28, 69, 153, 0.0, 0.0879664),
 (29, 69, 140, 0.0, 0.0828417),
 (30, 69, 141, 0.0, 0.0802044),
 (31, 71, 148, 0.0, 0.148458),
 (32, 71, 195, 0.0, 0.108432),
 (33, 71, 77, 0.0, 0.09